In [1]:
# inspired from Gaurav Duttas work in competition :  Binary Prediction of Poisonous Mushrooms

# 🚗 **2024 Kaggle Playground Series: Used Car Price Prediction** 🏁

Welcome to the 2024 Kaggle Playground Series! 🎉 In this competition, we will predict the price of used cars using various attributes. The goal is to create a robust model using **AutoGluon**, leveraging the power of GPUs for efficient computation. Let's dive in!

## 📚 **Libraries and Setup**

First, let's install and import the necessary libraries. We'll use AutoGluon for its ease of use and efficiency in model training and deployment.



In [2]:
!pip install ray==2.10.0 autogluon.tabular ipywidgets catboost==1.2.5 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
aiobotocore 2.13.2 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.29.165 which is incompatible.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.8.2 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatib

In [3]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import warnings
warnings.filterwarnings('ignore')

## 💾 **Load and Explore the Data**

Let's load the dataset and take a quick look at the first few rows

In [4]:
train=pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [5]:
train.duplicated().value_counts() # no duplicates :)

False    188533
Name: count, dtype: int64

## 🚀 **Training the Model with AutoGluon**

Now, we'll train the model using AutoGluon, which automatically handles feature engineering, model selection, and hyperparameter tuning. We'll utilize the GPU for faster training.

In [6]:
label = 'price'
train[label].describe()

count    1.885330e+05
mean     4.387802e+04
std      7.881952e+04
min      2.000000e+03
25%      1.700000e+04
50%      3.082500e+04
75%      4.990000e+04
max      2.954083e+06
Name: price, dtype: float64

In [7]:
predictor = TabularPredictor(label=label,eval_metric ='root_mean_squared_error',
                             problem_type="regression").fit(train,presets='best_quality',
                                                           time_limit=3600*11,verbosity=3,
                                                           excluded_model_types=['KNN'],
                                                           ag_args_fit={'num_gpus': 1}
                                                      )
results = predictor.fit_summary()

No path specified. Models will be saved in: "AutogluonModels/ag-20240910_171549"
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
GPU Count:          2
Memory Avail:       30.16 GB / 31.36 GB (96.2%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1},
 'auto_stack': True,
 'excluded_model_types': ['KNN'],
 'num_bag_sets': 1,
 'verbosity': 3}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': True,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging'

(_ray_fit pid=436) [50]	valid_set's rmse: 82322


(_ray_fit pid=493) 	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training. [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=437) [LightGBM] [Fatal] bin size 1665 cannot run on GPU
(_ray_fit pid=437) Warning: GPU mode might not be installed for LightGBM, GPU training raised an exception. Falling back to CPU training...Refer to LightGBM GPU documentation: https://github.com/Microsoft/LightGBM/tree/master/python-package#build-gpu-versionOne possible method is:	pip uninstall lightgbm -y	pip install lightgbm --install-option=--gpu
(_ray_fit pid=493) [LightGBM] [Fatal] bin size 1670 cannot run on GPU
(_ray_fit pid=493) Warning: GPU mode might not be installed for LightGBM, GPU training raised an exception. Falling back to CPU training...Refer to LightGBM GPU do

(_ray_fit pid=493) [50]	valid_set's rmse: 69880.3 [repeated 3x across cluster]


(_ray_fit pid=551) 	Training S1F5 with GPU, note that this may negatively impact model quality compared to CPU training. [repeated 2x across cluster]
(_ray_fit pid=520) [LightGBM] [Fatal] bin size 1666 cannot run on GPU
(_ray_fit pid=520) Warning: GPU mode might not be installed for LightGBM, GPU training raised an exception. Falling back to CPU training...Refer to LightGBM GPU documentation: https://github.com/Microsoft/LightGBM/tree/master/python-package#build-gpu-versionOne possible method is:	pip uninstall lightgbm -y	pip install lightgbm --install-option=--gpu
(_ray_fit pid=551) [LightGBM] [Fatal] bin size 1667 cannot run on GPU
(_ray_fit pid=551) Warning: GPU mode might not be installed for LightGBM, GPU training raised an exception. Falling back to CPU training...Refer to LightGBM GPU documentation: https://github.com/Microsoft/LightGBM/tree/master/python-package#build-gpu-versionOne possible method is:	pip uninstall lightgbm -y	pip install lightgbm --install-option=--gpu


...

(_ray_fit pid=10863) [0]	validation_0-rmse:80105.80584 [repeated 4x across cluster]


(_ray_fit pid=10861) /opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [19:56:59] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
(_ray_fit pid=10861) Potential solutions:
(_ray_fit pid=10861) - Use a data structure that matches the device ordinal in the booster.
(_ray_fit pid=10861) - Set the device for booster before call to inplace_predict.
(_ray_fit pid=10861) This warning will only be shown once.
(_ray_fit pid=10924) /opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [19:57:06] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0 [repeated 2x across cluster]
(_ray_fit pid=10924)   warnings.warn(smsg, UserWarning) [repeated 8x across cluster]
(_ray_fit pid=10924) /o

(_ray_fit pid=10924) [0]	validation_0-rmse:80501.17583 [repeated 5x across cluster]


(_ray_fit pid=10924) /opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [19:57:07] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
(_ray_fit pid=10924) Potential solutions:
(_ray_fit pid=10924) - Use a data structure that matches the device ordinal in the booster.
(_ray_fit pid=10924) - Set the device for booster before call to inplace_predict.
(_ray_fit pid=10924) This warning will only be shown once.
(_dystack pid=167) Saving AutogluonModels/ag-20240910_171549/ds_sub_fit/sub_fit_ho/models/XGBoost_BAG_L2/utils/oof.pkl
(_dystack pid=167) Saving AutogluonModels/ag-20240910_171549/ds_sub_fit/sub_fit_ho/models/XGBoost_BAG_L2/model.pkl
(_dystack pid=167) 	-73495.7824	 = Validation score   (-root_mean_squared_error)
(_dystack pid=167) 	27.81s	 = Training   runtime
(_dystack pi

*** Summary of fit() ***
Estimated performance of each model:
                          model     score_val              eval_metric  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           WeightedEnsemble_L3 -72460.423724  root_mean_squared_error     279.053290  20059.416847                0.003043           0.654773            3       True         85
1           WeightedEnsemble_L2 -72490.986540  root_mean_squared_error      95.635211   5099.747616                0.002633           0.792438            2       True         62
2               CatBoost_BAG_L2 -72498.486929  root_mean_squared_error     271.744857  19496.905839                0.657709          58.334367            2       True         66
3          CatBoost_r137_BAG_L2 -72517.432074  root_mean_squared_error     271.864494  19490.463722                0.777346          51.892251            2       True         81
4          CatBoost_r177_BAG_L2 -72530.828743  r

In [8]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-72460.423724,root_mean_squared_error,279.053290,20059.416847,0.003043,0.654773,3,True,85
1,WeightedEnsemble_L2,-72490.986540,root_mean_squared_error,95.635211,5099.747616,0.002633,0.792438,2,True,62
2,CatBoost_BAG_L2,-72498.486929,root_mean_squared_error,271.744857,19496.905839,0.657709,58.334367,2,True,66
3,CatBoost_r137_BAG_L2,-72517.432074,root_mean_squared_error,271.864494,19490.463722,0.777346,51.892251,2,True,81
4,CatBoost_r177_BAG_L2,-72530.828743,root_mean_squared_error,271.704902,19489.933062,0.617754,51.361590,2,True,72
...,...,...,...,...,...,...,...,...,...,...
80,NeuralNetTorch_r158_BAG_L1,-90209.459491,root_mean_squared_error,1.535660,444.695758,1.535660,444.695758,1,True,47
81,NeuralNetTorch_r22_BAG_L2,-90209.487958,root_mean_squared_error,276.094049,19806.412071,5.006902,367.840599,2,True,78
82,NeuralNetTorch_r22_BAG_L1,-90209.535136,root_mean_squared_error,1.299191,307.069428,1.299191,307.069428,1,True,16
83,NeuralNetTorch_r86_BAG_L1,-90209.558475,root_mean_squared_error,1.469808,326.626285,1.469808,326.626285,1,True,28


## 🏅 **Submission**

Finally, we'll generate the submission file. This file will be in the required format with id and price columns.

In [9]:
y_pred = predictor.predict(test)
y_pred[:5]

Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r128_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r137_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r13_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r167_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r177_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r49_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r50_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r69_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r70_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r86_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240910_171549/models/CatBoost_r9_BAG_L1/model.pkl
Loading: AutogluonModels/ag-2

0    18676.458984
1    85042.539062
2    58277.531250
3    30024.185547
4    29510.189453
Name: price, dtype: float32

In [10]:
df = pd.DataFrame(y_pred,columns=['price'])
df.head()

,price
0,18676.458984
1,85042.539062
2,58277.531250
3,30024.185547
4,29510.189453


In [11]:
sol=pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
sol.head()

,id,price
0,188533,43878.016
1,188534,43878.016
2,188535,43878.016
3,188536,43878.016
4,188537,43878.016


In [12]:
sol['price']=df['price']
sol.head()

,id,price
0,188533,18676.458984
1,188534,85042.539062
2,188535,58277.531250
3,188536,30024.185547
4,188537,29510.189453


In [13]:
sol.to_csv('submission.csv', index=False)

## 🎯 **Conclusion**

In this kernel, we successfully built a used car price prediction model using AutoGluon, efficiently utilizing GPU resources. Keep experimenting and refining your model to climb the leaderboard! 🏆